In [1]:
import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import DataLoader
import pandas as pd

In [2]:
# Read Data
spectrum_train = pd.read_excel('spectrum_train.xlsx')
spectrum_test = pd.read_excel('spectrum_valid.xlsx')
temp_train = pd.read_excel('temp_train.xlsx')
temp_test = pd.read_excel('temp_valid.xlsx')

In [3]:
spectrum_train_scaled = spectrum_train.multiply(10**12)
spectrum_test_scaled = spectrum_test.multiply(10**12)

In [4]:
# determine the supported device
def get_device():
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu') # don't have GPU 
    return device

# convert a df to tensor to be used in pytorch
def df_to_tensor(df):
    device = get_device()
    return torch.from_numpy(df.values).float().to(device)

In [5]:
input_data = df_to_tensor(temp_train)
output_data = df_to_tensor(spectrum_train_scaled)

In [6]:
input_data

tensor([[350.6424, 357.4240, 338.0000,  ..., 267.9975, 252.9999, 238.0000],
        [352.1875, 360.3629, 342.0451,  ..., 269.5426, 252.9999, 236.4549],
        [350.6424, 357.4240, 338.0000,  ..., 267.9975, 252.9999, 238.0000],
        ...,
        [597.3622, 585.6775, 574.4026,  ..., 493.7609, 505.0429, 515.4084],
        [595.8171, 582.7385, 570.3576,  ..., 492.2158, 505.0429, 516.9536],
        [601.9975, 594.4943, 586.5379,  ..., 498.3961, 505.0429, 510.7732]])

In [7]:
output_data

tensor([[ 0.1119,  0.0821,  0.0643,  ...,  0.3437,  0.3360,  0.3290],
        [ 0.1240,  0.0904,  0.0701,  ...,  0.3269,  0.3197,  0.3131],
        [ 0.1119,  0.0821,  0.0643,  ...,  0.3437,  0.3360,  0.3290],
        ...,
        [10.8134, 10.6301, 10.5865,  ..., 21.8142, 20.7956, 19.8604],
        [10.6221, 10.5147, 10.5277,  ..., 22.0530, 21.0220, 20.0754],
        [11.4288, 11.0081, 10.7881,  ..., 21.1053, 20.1236, 19.2221]])

In [8]:
class Net(nn.Module):
    def __init__(self, input_size, output_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 256)
        self.fc3 = nn.Linear(256, 512)
        self.fc4 = nn.Linear(512, 256)
        self.fc5 = nn.Linear(256, 128)
        self.fc6 = nn.Linear(128, output_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = torch.relu(self.fc5(x))
        x = self.fc6(x)
        return x

In [9]:
input_size = 11
output_size = 66
model = Net(input_size=input_size, output_size=output_size)

In [10]:
criterion = nn.MSELoss()  # Mean Squared Error loss for regression
optimizer = optim.Adam(model.parameters(), lr=0.00001)  # Adam optimizer

In [11]:
num_epochs = 15
batch_size = 1
num_batches = len(input_data) // batch_size

for epoch in range(num_epochs):
    for batch in range(num_batches):
        start = batch * batch_size
        end = start + batch_size

        inputs = input_data[start:end]
        targets = output_data[start:end]

        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        # Compute loss
        loss = criterion(outputs, targets)

        # Backpropagation and optimization
        loss.backward()
        optimizer.step()

    # Print the loss for this epoch
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.6f}")

Epoch 1/15, Loss: 1.132634
Epoch 2/15, Loss: 0.889480
Epoch 3/15, Loss: 0.440525
Epoch 4/15, Loss: 0.195986
Epoch 5/15, Loss: 0.206026
Epoch 6/15, Loss: 0.263091
Epoch 7/15, Loss: 0.285002
Epoch 8/15, Loss: 0.292297
Epoch 9/15, Loss: 0.307327
Epoch 10/15, Loss: 0.319834
Epoch 11/15, Loss: 0.335866
Epoch 12/15, Loss: 0.356722
Epoch 13/15, Loss: 0.390587
Epoch 14/15, Loss: 0.396699
Epoch 15/15, Loss: 0.416665


In [12]:
predictions = model(df_to_tensor(temp_train))

In [16]:
pd.DataFrame(predictions.detach().numpy(), columns=spectrum_test.columns).head()

,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,...,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008
0,2.904681,2.887842,3.061507,2.779536,2.297568,3.373097,3.314639,3.022262,3.131035,3.154243,...,7.350365,7.288757,7.283387,7.151779,7.005003,6.900735,7.020926,7.104105,6.962034,6.268903
1,2.816605,2.753494,2.972967,2.677421,2.132849,3.243225,3.110487,2.825399,2.980277,2.969292,...,7.124895,7.065416,7.018618,6.852043,6.730331,6.591538,6.762316,6.850696,6.760643,6.042833
2,2.904681,2.887842,3.061507,2.779536,2.297568,3.373097,3.314639,3.022262,3.131035,3.154243,...,7.350365,7.288757,7.283387,7.151779,7.005003,6.900735,7.020926,7.104105,6.962034,6.268903
3,2.456863,2.284718,2.646086,2.371874,1.765545,2.959602,2.580714,2.404639,2.631273,2.496146,...,6.393233,6.290544,6.251361,6.015221,5.978204,5.774881,5.918357,6.056137,6.088280,5.330615
4,2.839986,2.802385,2.963780,2.716825,2.207425,3.284668,3.225956,2.946444,3.034296,3.064332,...,7.216177,7.114171,7.112974,6.985060,6.839896,6.741595,6.834182,6.938564,6.791967,6.126687


In [17]:
spectrum_test_scaled.head()

,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,...,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008
0,2.373538,2.602481,2.775814,2.935397,3.074856,3.220465,3.363583,3.505908,3.647634,3.790090,...,10.069020,10.146292,10.208057,10.243674,10.246583,10.192531,10.036903,9.737349,9.326093,8.947765
1,2.027413,2.150030,2.250785,2.362463,2.467873,2.581041,2.694600,2.809533,2.925638,3.043688,...,8.794491,8.871859,8.934365,8.973221,8.983096,8.942860,8.813263,8.556884,8.201778,7.875021
2,2.470590,2.726463,2.921211,3.095970,3.246416,3.402639,3.555543,3.706972,3.857192,4.007671,...,10.460966,10.538142,10.599540,10.633985,10.634567,10.576088,10.412293,10.099323,9.670691,9.276407
3,1.950271,2.046041,2.131383,2.233903,2.333012,2.440313,2.548641,2.658761,2.770347,2.884035,...,8.542764,8.620148,8.682729,8.722118,8.733251,8.695631,8.571070,8.323129,7.979041,7.662406
4,2.523524,2.771409,2.957469,3.126776,3.273742,3.426844,3.576945,3.725913,3.874010,4.022686,...,10.456130,10.533535,10.595041,10.629530,10.630143,10.571716,10.408014,10.095198,9.666764,9.272662
